In [1]:
# Core MDAnalysis imports
import MDAnalysis as mda

# Interactive MD client for real-time data streaming
from imdclient.IMDREADER import IMDReader
from imdclient.tests.datafiles import NAMD_TOPOL  # Only if using test topology files

# Plotting and numerical libraries
import matplotlib.pyplot as plt
import numpy as np

# Logging for progress tracking
import logging

# Concurrency and threading for parallel processing and live plotting
import threading
import time  # Optional, for simulating real-time plot delays

In [ ]:
# Set up logging
logger = logging.getLogger("imdclient.IMDClient")
file_handler = logging.FileHandler("imdreader-distancedemo-ipynb.log")
formatter = logging.Formatter(
    "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)
logger.setLevel(logging.INFO)

# Enable widget for interactive plotting in Jupyter
%matplotlib widget

# Initialize Universe with IMD server connection
u = mda.Universe(NAMD_TOPOL, "imd://localhost:1029")  # Replace with your server if needed

# Create the initial plot
fig, ax = plt.subplots()
hl, = ax.plot([], [], color="blue")  # Empty plot to update dynamically
ax.set_xlim(0, 10)     # Set initial X limits
ax.set_ylim(0, 2)      # Set initial Y limits
ax.set_xlabel("Simulation Steps")
ax.set_ylabel("Distance between atoms 1 and 3")

update_frequency = 10  # Update the plot every 10 frames

# Function to update the line dynamically
def update_line(x_data, y_data, step, interval=50):
    hl.set_xdata(np.append(hl.get_xdata(), x_data))
    hl.set_ydata(np.append(hl.get_ydata(), y_data))
    
    # Update `xlim` every `interval` frames
    if step % interval == 0:
        ax.set_xlim(0, max(hl.get_xdata()) + 10)  # Extend by 10 for padding

    plt.draw()
    plt.pause(0.001)  # Pause to allow for the plot to update in Jupyter

# Function to read data and update plot
def read_data_from_buffer(update_interval=10):
    step = 0
    x_data, y_data = [], []  # Initialize empty lists for x and y data
    for ts in u.trajectory:
        # Calculate distance for each frame
        distance = np.linalg.norm(ts.positions[1][:2] - ts.positions[3][:2])
        x_data.append(step)  # Append the current step to x_data
        y_data.append(distance)  # Append the calculated distance to y_data
        step += 1
        
        # Only update the plot every `update_interval` frames
        if step % update_interval == 0:
            update_line(x_data, y_data, step)  # Plot the batch of collected points
            x_data, y_data = []  # Clear the lists after updating the plot

        # Log progress every 100 frames
        if step % 100 == 0:
            logger.info(f"Processed {step} frames")

    logger.info(f"Parsed {step} frames")

# Start a thread for data reading and plotting
data_thread = threading.Thread(target=read_data_from_buffer, args=(update_frequency,))
data_thread.daemon = True
data_thread.start()